### Section title placeholder

Description placeholder

In [ ]:
# !pip install -r requirements.txt
# !python -m spacy download de_core_news_sm

In [3]:
from warnings import filterwarnings

filterwarnings("ignore")

In [5]:
import os
import re
from typing import List, Dict
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

### Section title placeholder

Description placeholder

In [6]:
def get_html_text(filepath: str) -> str:
    """
    
    """
    with open(filepath, encoding='ISO-8859-1') as f:
        soup = BeautifulSoup(f.read(), "html")
    
    tags_to_decompose = [
        tag 
        for tag_name in ("style", "script")
        for tag in soup.find_all(tag_name)
    ]
    
    for tag in tags_to_decompose:
        tag.decompose()
    
    return soup.get_text(" ")

get_html_text(os.path.join("htmls", "0.html"))

'\n \n \n \n \n \n BauchspeicheldrÃ¼se | Klinik fÃ¼r Allgemein-, Viszeral- und Kinderchirurgie GÃ¶ttingen \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n Klinik fÃ¼r Allgemein-, Viszeral- und Kinderchirurgie \n zur Hauptnavigation springen \n zum Inhalt wechseln \n \n Aktuelles und Kontakt Kontakt \n \n \n Logo der UniverstÃ¤tsmedizin GÃ¶ttingen \n \n \n \n \n \n \n \n \n \n Navigation Ã¶ffnen oder schliessen \n \n \n \n \n \n \n Hauptnavigation Subnavigation Ã¶ffnen oder schliessen Medizinische Versorgung Poliklinik Sonographie SchilddrÃ¼se SpeiserÃ¶hre und Magen Darm BauchspeicheldrÃ¼se CED Leber und Galle Hernien Koloproktologie Adipositaschirurgie Kinderchirurgie Sarkomchirurgie HIPEC Roboterchirurgie InterdisziplinÃ¤re Zentren Subnavigation Ã¶ffnen oder schliessen Forschung Klinische Studien Tumorepigenetik AG Conradi AG Gaedcke AG Grade AG Krause AG Sperling AG Sprenger AG Wegwitz Promotion Publikationen Subnavigation Ã¶ffnen oder schliessen Lehre Module Blockpraktikum Famulaturen/P

Description placeholder

In [7]:
def get_all_texts() -> Dict[int, str]:
    """
    
    """
    return {
        int(file.replace(".html", "")): get_html_text(os.path.join("htmls", file))
        for file in os.listdir("htmls")
    }

all_texts = get_all_texts()

### Section title placeholder

Description placeholder

In [8]:
train_df = pd.read_csv("train.csv")
train_df

,url,doc_id,label
0,http://elbe-elster-klinikum.de/fachbereiche/ch...,1,1
1,http://klinikum-bayreuth.de/einrichtungen/zent...,3,3
2,http://klinikum-braunschweig.de/info.php/?id_o...,4,1
3,http://klinikum-braunschweig.de/info.php/?id_o...,5,1
4,http://klinikum-braunschweig.de/zuweiser/tumor...,6,3
...,...,...,...
95,http://www.unicross.uni-freiburg.de/thema/unifm/,140,1
96,http://www.uniklinik-duesseldorf.de/patienten-...,141,1
97,http://www.vivantes.de/fuer-sie-vor-ort/klinik...,144,2
98,http://www.vivantes.de/fuer-sie-vor-ort/klinik...,145,2


Description placeholder

In [9]:
train_df.label.value_counts()

2    59
1    32
3     9
Name: label, dtype: int64

Description placeholder

In [10]:
del train_df['url']
train_df['text'] = train_df['doc_id'].apply(lambda x: all_texts[x])
train_df

,doc_id,label,text
0,1,1,\n \n \n \n \n \n Elbe-Elster Klinikum - Chiru...
1,3,3,\n \n \n \n \n \n \n Onkologisches Zentrum - K...
2,4,1,\n \n Zentrum - SozialpÃ¤diatrisches Zentrum -...
3,5,1,\n \n Leistung - Spezielle UnterstÃ¼tzung bei ...
4,6,3,\n \n Zuweiser - Tumorkonferenzen - Tumorkonfe...
...,...,...,...
95,140,1,\n \n \n \n \n \n \n \n \n \n \n uniFM | uniCR...
96,141,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...
97,144,2,\n \n \n \n \n \n \n \n \n \n \n \n FÃ¼r Ãrzt...
98,145,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n Inne...


### Section title placeholder

Description placeholder

In [11]:
nlp = spacy.load("de_core_news_sm")

Description placeholder

In [12]:
def preprocessor(text: str) -> str:
    """
    Removes punctuation from text using regular expressions
    """
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    
    return text


def tokenize(sent: spacy.tokens.Span) -> List[str]:
    """
    Extracts a list of word lemmas from spacy's sentence object.
    Stopwords are filtered out as well as tokens representing only
    whitespace or punctuation.
    """
    lemmas = [
        preprocessor(token.lemma_)
        for token in sent
        if all([
            not token.is_stop,
            token.pos_ not in {"SPACE", "PUNCT"}
        ])
    ]
    return [lem for lem in lemmas if lem]


def tokenize_all(text: str) -> List[List[str]]:
    """
    Parses text to a list of lists. 
    Each list inside the main list represents a sentence in text.
    All tokens in the sentence are cleaned, lemmatized and filtered.
    """
    global nlp
    
    doc = nlp(text)
    
    return [
        tokenize(sent)
        for sent in doc.sents
    ]

Description placeholder

In [24]:
train_df['tokens'] = train_df['text'].apply(tokenize_all)
train_df['clean_sents'] = train_df['tokens'].apply(lambda x: [" ".join(toks) for toks in x])
train_df['clean_text'] = train_df['clean_sents'].apply(lambda x: " ".join(x))
train_df

,doc_id,label,text,sent_tokens,tokens,clean_sents,clean_text,vectors
0,1,1,\n \n \n \n \n \n Elbe-Elster Klinikum - Chiru...,"[[elbeelster, klinikum, chirurgie, finsterwald...","[[elbeelster, klinikum, chirurgie, finsterwald...",[elbeelster klinikum chirurgie finsterwalde su...,elbeelster klinikum chirurgie finsterwalde suc...,"[-0.3885404, 0.44329208, 1.1765691, 0.18398188..."
1,3,3,\n \n \n \n \n \n \n Onkologisches Zentrum - K...,"[[], [onkologisches, zentrum, klinikum, bayreu...","[[], [onkologisches, zentrum, klinikum, bayreu...","[, onkologisches zentrum klinikum bayreuth, ak...",onkologisches zentrum klinikum bayreuth aktue...,"[-0.05751672, 0.639646, 1.3166927, 0.35740948,..."
2,4,1,\n \n Zentrum - SozialpÃ¤diatrisches Zentrum -...,"[[], [zentrum, sozialpãdiatrisches, zentrum, s...","[[], [zentrum, sozialpãdiatrisches, zentrum, s...","[, zentrum sozialpãdiatrisches zentrum stãdtis...",zentrum sozialpãdiatrisches zentrum stãdtisch...,"[-0.28596362, 0.20628653, 0.94600266, 0.221883..."
3,5,1,\n \n Leistung - Spezielle UnterstÃ¼tzung bei ...,"[[], [leistung, speziell, unterstã¼tzung, anme...","[[], [leistung, speziell, unterstã¼tzung, anme...","[, leistung speziell unterstã¼tzung anmeldung ...",leistung speziell unterstã¼tzung anmeldung ge...,"[-0.33647257, 0.20716135, 1.0760807, 0.3270652..."
4,6,3,\n \n Zuweiser - Tumorkonferenzen - Tumorkonfe...,"[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[, zuweiser tumorkonferenzen tumorkonferenz ga...",zuweiser tumorkonferenzen tumorkonferenz gast...,"[-0.30618563, 0.48690826, 1.2848387, 0.2830735..."
...,...,...,...,...,...,...,...,...
95,140,1,\n \n \n \n \n \n \n \n \n \n \n uniFM | uniCR...,"[[unifm, unicross, news, and, magazin, theme, ...","[[unifm, unicross, news, and, magazin, theme, ...","[unifm unicross news and magazin theme home, k...",unifm unicross news and magazin theme home kon...,"[-0.30736154, 0.25532013, 0.8725422, 0.3323875..."
96,141,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,"[[], [interdisziplinãre, neurovaskulãre, konfe...","[[], [interdisziplinãre, neurovaskulãre, konfe...","[, interdisziplinãre neurovaskulãre konferenz ...",interdisziplinãre neurovaskulãre konferenz ǀ ...,"[-0.5920495, 0.3950545, 1.3128979, 0.16924097,..."
97,144,2,\n \n \n \n \n \n \n \n \n \n \n \n FÃ¼r Ãrzt...,"[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[fã¼r, ãrzte, vivantes javascript scheinen bro...",fã¼r ãrzte vivantes javascript scheinen browse...,"[-0.5012306, 0.24726734, 1.4206642, 0.10274181..."
98,145,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n Inne...,"[[innere, medizin, â, hãmatologie, onkologie, ...","[[innere, medizin, â, hãmatologie, onkologie, ...",[innere medizin â hãmatologie onkologie pallia...,innere medizin â hãmatologie onkologie palliat...,"[-0.50758284, 0.15702526, 1.3736418, 0.1842688..."


### Section title placeholder

Description placeholder

In [25]:
train_df['vectors'] = train_df['clean_text'].apply(lambda x: nlp(x).vector)
train_df

,doc_id,label,text,sent_tokens,tokens,clean_sents,clean_text,vectors
0,1,1,\n \n \n \n \n \n Elbe-Elster Klinikum - Chiru...,"[[elbeelster, klinikum, chirurgie, finsterwald...","[[elbeelster, klinikum, chirurgie, finsterwald...",[elbeelster klinikum chirurgie finsterwalde su...,elbeelster klinikum chirurgie finsterwalde suc...,"[-0.3885404, 0.44329208, 1.1765691, 0.18398188..."
1,3,3,\n \n \n \n \n \n \n Onkologisches Zentrum - K...,"[[], [onkologisches, zentrum, klinikum, bayreu...","[[], [onkologisches, zentrum, klinikum, bayreu...","[, onkologisches zentrum klinikum bayreuth, ak...",onkologisches zentrum klinikum bayreuth aktue...,"[-0.05751672, 0.639646, 1.3166927, 0.35740948,..."
2,4,1,\n \n Zentrum - SozialpÃ¤diatrisches Zentrum -...,"[[], [zentrum, sozialpãdiatrisches, zentrum, s...","[[], [zentrum, sozialpãdiatrisches, zentrum, s...","[, zentrum sozialpãdiatrisches zentrum stãdtis...",zentrum sozialpãdiatrisches zentrum stãdtisch...,"[-0.28596362, 0.20628653, 0.94600266, 0.221883..."
3,5,1,\n \n Leistung - Spezielle UnterstÃ¼tzung bei ...,"[[], [leistung, speziell, unterstã¼tzung, anme...","[[], [leistung, speziell, unterstã¼tzung, anme...","[, leistung speziell unterstã¼tzung anmeldung ...",leistung speziell unterstã¼tzung anmeldung ge...,"[-0.33647257, 0.20716135, 1.0760807, 0.3270652..."
4,6,3,\n \n Zuweiser - Tumorkonferenzen - Tumorkonfe...,"[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[, zuweiser tumorkonferenzen tumorkonferenz ga...",zuweiser tumorkonferenzen tumorkonferenz gast...,"[-0.30618563, 0.48690826, 1.2848387, 0.2830735..."
...,...,...,...,...,...,...,...,...
95,140,1,\n \n \n \n \n \n \n \n \n \n \n uniFM | uniCR...,"[[unifm, unicross, news, and, magazin, theme, ...","[[unifm, unicross, news, and, magazin, theme, ...","[unifm unicross news and magazin theme home, k...",unifm unicross news and magazin theme home kon...,"[-0.30736154, 0.25532013, 0.8725422, 0.3323875..."
96,141,1,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,"[[], [interdisziplinãre, neurovaskulãre, konfe...","[[], [interdisziplinãre, neurovaskulãre, konfe...","[, interdisziplinãre neurovaskulãre konferenz ...",interdisziplinãre neurovaskulãre konferenz ǀ ...,"[-0.5920495, 0.3950545, 1.3128979, 0.16924097,..."
97,144,2,\n \n \n \n \n \n \n \n \n \n \n \n FÃ¼r Ãrzt...,"[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[fã¼r, ãrzte, vivantes javascript scheinen bro...",fã¼r ãrzte vivantes javascript scheinen browse...,"[-0.5012306, 0.24726734, 1.4206642, 0.10274181..."
98,145,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n Inne...,"[[innere, medizin, â, hãmatologie, onkologie, ...","[[innere, medizin, â, hãmatologie, onkologie, ...",[innere medizin â hãmatologie onkologie pallia...,innere medizin â hãmatologie onkologie palliat...,"[-0.50758284, 0.15702526, 1.3736418, 0.1842688..."


Description placeholder

In [26]:
class_3_oversample = train_df[train_df.label == 3].sample(20, replace=True)
oversampled_train_df = pd.concat([train_df[train_df.label != 3], class_3_oversample])
oversampled_train_df.label.value_counts()

2    59
1    32
3    20
Name: label, dtype: int64

In [27]:
x, y = np.stack(oversampled_train_df.vectors), oversampled_train_df.label.to_numpy()

X_train, x_val, Y_train, y_val = train_test_split(x, y, train_size=0.9, shuffle=True, random_state=1, stratify=y)

In [28]:
def eval_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    
    acc = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    
    print(f'Accuracy {acc}\n')
    print(conf_matrix)
    print()
    print(report)

In [53]:
lgr = LogisticRegression(n_jobs=5, C=1, class_weight="balanced")
lgr.fit(X_train, Y_train)

eval_model(lgr, x_val, y_val)

Accuracy 0.8333333333333334

[[3 1 0]
 [1 5 0]
 [0 0 2]]

              precision    recall  f1-score   support

           1       0.75      0.75      0.75         4
           2       0.83      0.83      0.83         6
           3       1.00      1.00      1.00         2

   micro avg       0.83      0.83      0.83        12
   macro avg       0.86      0.86      0.86        12
weighted avg       0.83      0.83      0.83        12



In [45]:
sgd = SGDClassifier(penalty='l2', alpha=1e-3, random_state=0, max_iter=10000)
sgd.fit(X_train, Y_train)

eval_model(sgd, x_val, y_val)

Accuracy 0.8333333333333334

[[4 0 0]
 [2 4 0]
 [0 0 2]]

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         4
           2       1.00      0.67      0.80         6
           3       1.00      1.00      1.00         2

   micro avg       0.83      0.83      0.83        12
   macro avg       0.89      0.89      0.87        12
weighted avg       0.89      0.83      0.83        12



In [46]:
Y_train_bin = np.array([2 if l == 3 else l for l in Y_train])
y_val_bin = np.array([2 if l == 3 else l for l in y_val])

In [47]:
sgd_bin = SGDClassifier(penalty='l2', alpha=1e-3, random_state=0, max_iter=10000, tol=None)
sgd_bin.fit(X_train, Y_train_bin)

eval_model(sgd_bin, x_val, y_val_bin)

Accuracy 0.8333333333333334

[[4 0]
 [2 6]]

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         4
           2       1.00      0.75      0.86         8

   micro avg       0.83      0.83      0.83        12
   macro avg       0.83      0.88      0.83        12
weighted avg       0.89      0.83      0.84        12



In [22]:
kw2tt = pd.read_csv("keyword2tumor_type.csv")
kw2tt

,keyword,tumor_type
0,senologische,Brust
1,brustzentrum,Brust
2,breast,Brust
3,thorax,Brust
4,thorakale,Brust
...,...,...
121,kinderonkologie,Pädiatrische
122,pädiatrische,Pädiatrische
123,schwerpunkt,Schwerpunkt
124,stammzelltransplantation,Stammzelltransplantation


In [83]:
from collections import Counter, defaultdict

In [95]:
def detect_type(tokens):
    global kw2tt
    
    flat_tokens = [t for toks in tokens for t in toks]
    keywords = set(flat_tokens).intersection(set(kw2tt.keyword))
    word_counts = Counter(flat_tokens)
    type_map = dict(zip(kw2tt.keyword, kw2tt.tumor_type))
    
    type_counts = defaultdict(int)
    for kw in keywords:
        type_ = type_map[kw]
        type_counts[type_] += word_counts[kw]
    
    most_freq_type, _ = max(type_counts.items(), key=lambda x: x[1])
    
    return most_freq_type

In [97]:
train_df_23 = train_df[train_df.label.apply(lambda x: x in {2, 3})]

train_df_23['tumor_type'] = train_df_23['tokens'].apply(detect_type)
train_df_23

,doc_id,label,text,tokens,clean_sents,clean_text,vectors,tumor_type
1,3,3,\n \n \n \n \n \n \n Onkologisches Zentrum - K...,"[[], [onkologisches, zentrum, klinikum, bayreu...","[, onkologisches zentrum klinikum bayreuth, ak...",onkologisches zentrum klinikum bayreuth aktue...,"[-0.05751672, 0.639646, 1.3166927, 0.35740948,...",Brust
4,6,3,\n \n Zuweiser - Tumorkonferenzen - Tumorkonfe...,"[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[, zuweiser tumorkonferenzen tumorkonferenz ga...",zuweiser tumorkonferenzen tumorkonferenz gast...,"[-0.30618563, 0.48690826, 1.2848387, 0.2830735...",Magen
10,13,2,\n \n \n Krebszentrum Reutlingen: Zentrale Ein...,"[[], [krebszentrum, reutlingen, zentrale, einr...","[, krebszentrum reutlingen zentrale einrichtun...",krebszentrum reutlingen zentrale einrichtung ...,"[0.084352665, 0.26025638, 1.3280164, 0.2644429...",Interdisziplinär
11,14,2,\n \n \n \n \n Leistungsspektrum: Sankt Marien...,"[[], [leistungsspektrum], [sankt, marienhospit...","[, leistungsspektrum, sankt marienhospital bue...",leistungsspektrum sankt marienhospital buer g...,"[-0.57697684, 0.012666922, 1.0382532, 0.446892...",Darm
12,17,2,\n \n \n \n \n Minimal-invasive Chirurgie: San...,"[[minimalinvasive, chirurgie], [sankt, marienh...","[minimalinvasive chirurgie, sankt marienhospit...",minimalinvasive chirurgie sankt marienhospital...,"[-0.641403, 0.026014825, 1.0859053, 0.44600007...",Darm
...,...,...,...,...,...,...,...,...
89,132,2,\n \n \n \n \n Thorax Klinik: Lungenkrebszentr...,"[[thorax, klinik, lungenkrebszentrum, nct, hei...",[thorax klinik lungenkrebszentrum nct heidelbe...,thorax klinik lungenkrebszentrum nct heidelber...,"[-0.42971414, -0.065575615, 0.7600092, 0.37320...",Lunge
90,133,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n A...,"[[], [ambulanz, sprechstundenund, kontakt, uni...","[, ambulanz sprechstundenund kontakt universit...",ambulanz sprechstundenund kontakt universitãt...,"[-0.6437655, 0.15264697, 1.2336291, 0.27001214...",Schwerpunkt
91,136,2,\n \n \n \n \n \n \n \n Veranstaltungen: UKM -...,"[[], [veranstaltung, ukm, hautklinik], [suche,...","[, veranstaltung ukm hautklinik, suche suche h...",veranstaltung ukm hautklinik suche suche haut...,"[-0.21947816, 0.9361462, 1.2044301, 0.11974818...",Haut
97,144,2,\n \n \n \n \n \n \n \n \n \n \n \n FÃ¼r Ãrzt...,"[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[fã¼r, ãrzte, vivantes javascript scheinen bro...",fã¼r ãrzte vivantes javascript scheinen browse...,"[-0.5012306, 0.24726734, 1.4206642, 0.10274181...",Schwerpunkt


In [113]:
weekdays = {'sonntag', 'montag', 'dienstag', 'mittwoch', 'donnerstag', 'freitag', 'samstag'}

def detect_weekday(tokens):
    flat_tokens = [t for toks in tokens for t in toks]
    found_weekdays = set(flat_tokens).intersection(weekdays)
    word_counts = Counter(flat_tokens)
    
    weekday_counts = defaultdict(int)
    for wd in found_weekdays:
        weekday_counts[wd] += word_counts[wd]
    
    if not weekday_counts:
        return None
    
    most_freq_wd, _ = max(weekday_counts.items(), key=lambda x: x[1])
    
    return most_freq_wd

In [114]:
train_df_23['weekday'] = train_df_23['tokens'].apply(detect_weekday)
train_df_23

,doc_id,label,text,tokens,clean_sents,clean_text,vectors,tumor_type,weekday
1,3,3,\n \n \n \n \n \n \n Onkologisches Zentrum - K...,"[[], [onkologisches, zentrum, klinikum, bayreu...","[, onkologisches zentrum klinikum bayreuth, ak...",onkologisches zentrum klinikum bayreuth aktue...,"[-0.05751672, 0.639646, 1.3166927, 0.35740948,...",Brust,donnerstag
4,6,3,\n \n Zuweiser - Tumorkonferenzen - Tumorkonfe...,"[[], [zuweiser, tumorkonferenzen, tumorkonfere...","[, zuweiser tumorkonferenzen tumorkonferenz ga...",zuweiser tumorkonferenzen tumorkonferenz gast...,"[-0.30618563, 0.48690826, 1.2848387, 0.2830735...",Magen,dienstag
10,13,2,\n \n \n Krebszentrum Reutlingen: Zentrale Ein...,"[[], [krebszentrum, reutlingen, zentrale, einr...","[, krebszentrum reutlingen zentrale einrichtun...",krebszentrum reutlingen zentrale einrichtung ...,"[0.084352665, 0.26025638, 1.3280164, 0.2644429...",Interdisziplinär,None
11,14,2,\n \n \n \n \n Leistungsspektrum: Sankt Marien...,"[[], [leistungsspektrum], [sankt, marienhospit...","[, leistungsspektrum, sankt marienhospital bue...",leistungsspektrum sankt marienhospital buer g...,"[-0.57697684, 0.012666922, 1.0382532, 0.446892...",Darm,None
12,17,2,\n \n \n \n \n Minimal-invasive Chirurgie: San...,"[[minimalinvasive, chirurgie], [sankt, marienh...","[minimalinvasive chirurgie, sankt marienhospit...",minimalinvasive chirurgie sankt marienhospital...,"[-0.641403, 0.026014825, 1.0859053, 0.44600007...",Darm,None
...,...,...,...,...,...,...,...,...,...
89,132,2,\n \n \n \n \n Thorax Klinik: Lungenkrebszentr...,"[[thorax, klinik, lungenkrebszentrum, nct, hei...",[thorax klinik lungenkrebszentrum nct heidelbe...,thorax klinik lungenkrebszentrum nct heidelber...,"[-0.42971414, -0.065575615, 0.7600092, 0.37320...",Lunge,None
90,133,2,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n A...,"[[], [ambulanz, sprechstundenund, kontakt, uni...","[, ambulanz sprechstundenund kontakt universit...",ambulanz sprechstundenund kontakt universitãt...,"[-0.6437655, 0.15264697, 1.2336291, 0.27001214...",Schwerpunkt,None
91,136,2,\n \n \n \n \n \n \n \n Veranstaltungen: UKM -...,"[[], [veranstaltung, ukm, hautklinik], [suche,...","[, veranstaltung ukm hautklinik, suche suche h...",veranstaltung ukm hautklinik suche suche haut...,"[-0.21947816, 0.9361462, 1.2044301, 0.11974818...",Haut,mittwoch
97,144,2,\n \n \n \n \n \n \n \n \n \n \n \n FÃ¼r Ãrzt...,"[[fã¼r], [ãrzte], [vivantes, javascript, schei...","[fã¼r, ãrzte, vivantes javascript scheinen bro...",fã¼r ãrzte vivantes javascript scheinen browse...,"[-0.5012306, 0.24726734, 1.4206642, 0.10274181...",Schwerpunkt,donnerstag


In [118]:
train_df_23[~train_df_23.weekday.isna()].shape

(17, 9)